In [5]:
import pandas as pd
import os
import cv2 as cv
import numpy as np
from shapely.geometry import Polygon

In [6]:
bb_df = pd.read_csv("../bounding_box_data.csv", sep=",")
bb_df.head(5)

,deviceID,timestamp,x,y,width,height
0,0a:92:5e:29:ec:a2,20191225T000100+0900,660,388,125,125
1,0a:92:5e:29:ec:a2,20191225T000606+0900,660,388,125,125
2,0a:92:5e:29:ec:a2,20191225T001111+0900,660,388,125,125
3,0a:92:5e:29:ec:a2,20191225T001617+0900,660,388,125,125
4,0a:92:5e:29:ec:a2,20191225T002120+0900,660,388,125,125


In [73]:
r = ["true_pos", "false_pos", "false_neg", "precision", "recall"]
t = ["0.10", "0.50", "0.75"]
columns = []
for j in t:
    for h in r:
        columns.append(h + "_" + j)

model_names = []
for t in ["s", "x"]:
    for i in range(18):
        model_names.append("YOLOv5" + t + "_" + str(i).zfill(2))

performance_df = pd.DataFrame(data={"model":model_names}, columns=["model"] + columns)
performance_df.head(5)

,model,true_pos_0.10,false_pos_0.10,false_neg_0.10,precision_0.10,recall_0.10,true_pos_0.50,false_pos_0.50,false_neg_0.50,precision_0.50,recall_0.50,true_pos_0.75,false_pos_0.75,false_neg_0.75,precision_0.75,recall_0.75
0,YOLOv5s_00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,YOLOv5s_01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,YOLOv5s_02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,YOLOv5s_03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,YOLOv5s_04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
iou_df = pd.DataFrame(data={"YOLOv5s_00":[np.NAN]*1100}, columns=model_names)
iou_df.head(5)

,YOLOv5s_00,YOLOv5s_01,YOLOv5s_02,YOLOv5s_03,YOLOv5s_04,YOLOv5s_05,YOLOv5s_06,YOLOv5s_07,YOLOv5s_08,YOLOv5s_09,...,YOLOv5x_08,YOLOv5x_09,YOLOv5x_10,YOLOv5x_11,YOLOv5x_12,YOLOv5x_13,YOLOv5x_14,YOLOv5x_15,YOLOv5x_16,YOLOv5x_17
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
# Bounding boxes supplied in format: [x, y, width, height]
def calc_iou(bbA, bbB):                   #top L     top R     bottom R   bottom L
    # Bounding boxes converted to format: [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]
    bbA = Polygon([[bbA[0], bbA[1]], [bbA[0]+bbA[2], bbA[1]], [bbA[0]+bbA[2], bbA[1]-bbA[3]], [bbA[0], bbA[1]-bbA[3]]])
    bbB = Polygon([[bbB[0], bbB[1]], [bbB[0]+bbB[2], bbB[1]], [bbB[0]+bbB[2], bbB[1]-bbB[3]], [bbB[0], bbB[1]-bbB[3]]])
    
    iou = bbA.intersection(bbB).area / bbA.union(bbB).area
    return iou

In [92]:
thres = ["0.10", "0.50", "0.75"]
test_pics = os.listdir("YOLOv5 Master/test/images")
all_timestamps = [test_pics[i].split("_")[0].split("-")[0] + "+0900" for i in range(len(test_pics))]

for model in model_names:
    print("\t###", model, "###")
    
    pics = os.listdir("detect/labels/" + model)
    ious = []

    # Get timestamps for images where current model have identified weighers
    pos_pics = [pics[i].split("_")[0].split("-")[0] + "+0900" for i in range(len(pics))]
    # Get timestamps for images where current model haven't identified a weigher
    neg_pics = list(set(all_timestamps) - set(pos_pics))
    # Count false negatives
    false_neg = 0
    for neg in neg_pics:
        nx = bb_df.loc[bb_df["timestamp"] == neg, "x"].item()
        ny = bb_df.loc[bb_df["timestamp"] == neg, "y"].item()
        if not (nx == 0 and ny == 0):
            false_neg += 1

    for i in range(len(pics)):
        if not pics[i].endswith(".txt"):
            continue
        else:
            time_stamp = pics[i].split("_")[0].split("-")[0] + "+0900"
            # gtx = round(bb_df.loc[bb_df["timestamp"] == time_stamp, "x"].item()*0.5)
            # gty = round(bb_df.loc[bb_df["timestamp"] == time_stamp, "y"].item()*0.88888)
            # gtw = round(bb_df.loc[bb_df["timestamp"] == time_stamp, "width"].item()*0.5)
            # gth = round(bb_df.loc[bb_df["timestamp"] == time_stamp, "height"].item()*0.5)
            # Get ground truth bounding box from original dataset labels
            f = open("YOLOv5 Master/test/labels/" + pics[i], "r")
            contents = f.readlines()
            f.close()
            gtxc, gtyc, gtw, gth = [round(float(num)*640) for num in contents[0].split()[1:5]]
            gtx = gtxc - round((gtw/2))
            gty = gtyc - round((gth/2))
            gt = [gtx, gty, gtw, gth]

            f = open("detect/labels/" + model + "/" + pics[i], "r")
            contents = f.readlines()
            f.close()

            for j in range(len(contents)):
                # Get predicted bounding box coordinates
                xc, yc, w, h = [round(float(num)*640) for num in contents[j].split()[1:5]]
                x = xc - round((w/2))
                y = yc - round((h/2))
                iou = calc_iou(gt, [x, y, w, h])
                ious.append(iou)
                # print(gt, "\t", x, y, w, h)
                # print(iou)
                # img = cv.imread("detect/" + model + "/" + pics[i].split("txt")[0] + "jpg")
                # cv.rectangle(img=img, pt1=(x, y), pt2=(x+w, y+h), color=(255, 0, 0), thickness=1)
                # cv.rectangle(img=img, pt1=(gtx, gty), pt2=(gtx+gtw, gty+gth), color=(0, 255, 0), thickness=1)
                # cv.imshow("e", img)
                # k = cv.waitKey(0)
                # print(k)
                # cv.destroyWindow("e")
                # cv.waitKey(1)
                # if k == 113:
                #     break
            iou_df.loc[0:len(ious), model] = pd.Series(ious)
    
    for k in range(len(thres)):
        true_pos = 0
        false_pos = 0   
        for val in ious:
            if val >= float(thres[k]):
                true_pos += 1
            elif val < float(thres[k]):
                false_pos += 1
            
        # print("\tTrue positives:\t\t", true_pos)
        # print("\tFalse positives:\t", false_pos)
        # print("\tFalse negatives:\t", false_neg)

        performance_df.loc[performance_df["model"] == model, "true_pos_" + thres[k]] = true_pos
        performance_df.loc[performance_df["model"] == model, "false_pos_" + thres[k]] = false_pos
        performance_df.loc[performance_df["model"] == model, "false_neg_" + thres[k]] = false_neg
        performance_df.loc[performance_df["model"] == model, "precision_" + thres[k]] = true_pos / (true_pos+false_pos)
        performance_df.loc[performance_df["model"] == model, "recall_" + thres[k]] = true_pos / (true_pos+false_neg)

        

	### YOLOv5s_00 ###
	### YOLOv5s_01 ###
	### YOLOv5s_02 ###
	### YOLOv5s_03 ###
	### YOLOv5s_04 ###
	### YOLOv5s_05 ###
	### YOLOv5s_06 ###
	### YOLOv5s_07 ###
	### YOLOv5s_08 ###
	### YOLOv5s_09 ###
	### YOLOv5s_10 ###
	### YOLOv5s_11 ###
	### YOLOv5s_12 ###
	### YOLOv5s_13 ###
	### YOLOv5s_14 ###
	### YOLOv5s_15 ###
	### YOLOv5s_16 ###
	### YOLOv5s_17 ###
	### YOLOv5x_00 ###
	### YOLOv5x_01 ###
	### YOLOv5x_02 ###
	### YOLOv5x_03 ###
	### YOLOv5x_04 ###
	### YOLOv5x_05 ###
	### YOLOv5x_06 ###
	### YOLOv5x_07 ###
	### YOLOv5x_08 ###
	### YOLOv5x_09 ###
	### YOLOv5x_10 ###
	### YOLOv5x_11 ###
	### YOLOv5x_12 ###
	### YOLOv5x_13 ###
	### YOLOv5x_14 ###
	### YOLOv5x_15 ###
	### YOLOv5x_16 ###
	### YOLOv5x_17 ###


In [7]:
test_pics = os.listdir("YOLOv5 Master/test/images")
all_timestamps = [test_pics[i].split("_")[0].split("-")[0] + "+0900" for i in range(len(test_pics))]

# # Get timestamps for images where current model have identified weighers
#     pos_pics = [pics[i].split("_")[0].split("-")[0] + "+0900" for i in range(len(pics))]
#     # Get timestamps for images where current model haven't identified a weigher
#     neg_pics = list(set(all_timestamps) - set(pos_pics))
#     # Count false negatives
#     false_neg = 0
#     for neg in neg_pics:
#         nx = bb_df.loc[bb_df["timestamp"] == neg, "x"].item()
#         ny = bb_df.loc[bb_df["timestamp"] == neg, "y"].item()
#         if not (nx == 0 and ny == 0):
#             false_neg += 1

In [9]:
total = 0
true_neg = 0
for i in range(len(all_timestamps)):
    total += 1
    fx = bb_df.loc[bb_df["timestamp"] == all_timestamps[i], "x"].item()
    fy = bb_df.loc[bb_df["timestamp"] == all_timestamps[i], "y"].item()
    if (fx == 0) and (fy == 0):
        true_neg += 1
print( total, true_neg)


850 5


In [96]:
performance_df.to_csv(path_or_buf="yolo_performance.csv", index=False)
iou_df = iou_df.dropna(how="all")
iou_df.to_csv(path_or_buf="iou_df.csv", index=False)

In [90]:
performance_df.head()

,model,true_pos_0.10,false_pos_0.10,false_neg_0.10,precision_0.10,recall_0.10,true_pos_0.50,false_pos_0.50,false_neg_0.50,precision_0.50,recall_0.50,true_pos_0.75,false_pos_0.75,false_neg_0.75,precision_0.75,recall_0.75
0,YOLOv5s_00,845,1,0,0.998818,1.0,844,2,0,0.997636,1.0,843,3,0,0.996454,1.0
1,YOLOv5s_01,845,2,0,0.997639,1.0,844,3,0,0.996458,1.0,844,3,0,0.996458,1.0
2,YOLOv5s_02,845,1,0,0.998818,1.0,844,2,0,0.997636,1.0,844,2,0,0.997636,1.0
3,YOLOv5s_03,844,2,1,0.997636,0.998817,844,2,1,0.997636,0.998817,844,2,1,0.997636,0.998817
4,YOLOv5s_04,844,1,1,0.998817,0.998817,844,1,1,0.998817,0.998817,844,1,1,0.998817,0.998817


In [98]:
iou_df.head()

,YOLOv5s_00,YOLOv5s_01,YOLOv5s_02,YOLOv5s_03,YOLOv5s_04,YOLOv5s_05,YOLOv5s_06,YOLOv5s_07,YOLOv5s_08,YOLOv5s_09,...,YOLOv5x_08,YOLOv5x_09,YOLOv5x_10,YOLOv5x_11,YOLOv5x_12,YOLOv5x_13,YOLOv5x_14,YOLOv5x_15,YOLOv5x_16,YOLOv5x_17
0,0.984127,0.968254,0.968254,1.0,0.968254,1.0,0.968254,1.0,0.984127,0.953125,...,1.0,1.0,1.0,1.0,1.0,0.984127,1.0,0.984127,0.806876,0.895303
1,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.952381,0.968254,0.953125,...,0.968254,1.0,1.0,1.0,0.968254,0.968254,0.968254,0.968254,0.832394,0.852736
2,1.000000,1.0,0.984127,1.0,1.0,1.0,1.0,1.0,0.968254,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.923554,0.890219,0.923077
3,1.000000,1.0,0.968254,1.0,0.968254,0.968254,0.968254,0.968254,0.968254,0.952881,...,0.968254,1.0,1.0,0.968254,0.984127,0.968254,0.968254,0.880166,0.697173,0.880626
4,0.908869,0.852739,0.90864,0.923077,0.923077,0.90864,0.923077,0.90864,0.922844,0.894866,...,0.923077,0.953125,0.937988,0.894866,0.923077,0.90864,0.852739,0.907945,0.694458,0.953125


In [19]:
contents = "0 0.5359375 0.5515625 0.096875 0.096875"
gtxc, gtyc, gtw, gth = [round(float(num)*640) for num in contents.split()[1:]]
gtx = gtxc - round((gtw/2))
gty = gtyc - round((gth/2))
gt = [gtx, gty, gtw, gth]
gt

[312, 322, 62, 62]

In [26]:
# 625,364,125,125
img = cv.imread("YOLOv5 Master/test/images/20200109T122805-0900_infrared_image_png.rf.9bf972fa8fa919bac8b72a2a8f530cd8.jpg")
img = cv.rectangle(img, pt1=(312, 322), pt2=(374, 384), color=(0, 255, 0), thickness=2)
cv.imshow("test", img)
cv.waitKey(0)
cv.destroyWindow("test")
cv.waitKey(1)

-1